In [205]:
import pandas as pd
import requests
import json
base_url = 'https://aqicn.org/data-platform/covid19/report/15408-c5ae2edd/'
req = ['2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020']
file_names = []

def api_call_csv(timestamp):
    url = base_url + timestamp
    response = requests.get(url)
    file_path = "../data/"
    file_name = f"api-download-pollution-{timestamp}.csv"
    csv_file = open(file_path + file_name, 'wb')
    csv_file.write(response.content)
    csv_file.close()
    file_names.append(file_path + file_name)
    return file_names

def file_instantiater():
    for year in req:
        api_call_csv(year)
        
file_instantiater()

print(file_names)

['../data/api-download-pollution-2019Q1.csv', '../data/api-download-pollution-2019Q2.csv', '../data/api-download-pollution-2019Q3.csv', '../data/api-download-pollution-2019Q4.csv', '../data/api-download-pollution-2020.csv']


In [206]:
def dataframe_converter(list_of_file_names):
    dataframe_list = []
    for file in list_of_file_names:
        dataframe_list.append(pd.read_csv(file, skiprows=4))
    return dataframe_list

list_of_dframes = dataframe_converter(file_names)


        
        # pd.read_csv("../data/api-download-pollution-2019Q1.csv", skiprows=4)

In [426]:
def dataframe_cleaner(list_of_dframes):
    #converted_columns_datetime = []
    #for dataframe in list_of_dframes:
    #    dataframe[["Date"]] = pd.to_datetime(dataframe["Date"])
    #    print(dataframe["Date"])
    #list_of_dframes = Q12019, Q22019, Q32019, Q42019, Q2020
    list_of_dframes[0]["Date"] = pd.to_datetime(list_of_dframes[0]["Date"])
    list_of_dframes[1]["Date"] = pd.to_datetime(list_of_dframes[1]["Date"])
    list_of_dframes[2]["Date"] = pd.to_datetime(list_of_dframes[2]["Date"])
    list_of_dframes[3]["Date"] = pd.to_datetime(list_of_dframes[3]["Date"])
    list_of_dframes[4]["Date"] = pd.to_datetime(list_of_dframes[4]["Date"])
    return list_of_dframes

list_of_dframes = dataframe_cleaner(list_of_dframes)

Q12019 = list_of_dframes[0][["Date", "Country", "City", "Specie", "median"]]
Q22019 = list_of_dframes[1][["Date", "Country", "City", "Specie", "median"]]
Q32019 = list_of_dframes[2][["Date", "Country", "City", "Specie", "median"]]
Q42019 = list_of_dframes[3][["Date", "Country", "City", "Specie", "median"]]
Q2020 = list_of_dframes[4][["Date", "Country", "City", "Specie", "median"]]
    

#Q12019["Date"] = pd.to_datetime(Q12019["Date"])

In [428]:
Q22019.dtypes

Date       datetime64[ns]
Country            object
City               object
Specie             object
median            float64
dtype: object

In [419]:
Q12019.dtypes

Date       datetime64[ns]
Country            object
City               object
Specie             object
median            float64
dtype: object

In [429]:
Q12019

,Date,Country,City,Specie,median
0,2019-01-16,AE,Abu Dhabi,pm10,97.0
1,2019-01-22,AE,Abu Dhabi,pm10,55.0
2,2019-01-26,AE,Abu Dhabi,pm10,160.0
3,2019-01-07,AE,Abu Dhabi,pm10,72.0
4,2019-01-10,AE,Abu Dhabi,pm10,87.0
...,...,...,...,...,...
491156,2019-02-22,RS,Belgrade,o3,10.5
491157,2019-03-15,RS,Belgrade,o3,16.8
491158,2018-12-31,RS,Belgrade,o3,6.6
491159,2019-03-11,RS,Belgrade,o3,19.1


In [430]:
pollution_types = ["o3", "pm25", "pm10", "no2"]

def get_pollution_data(city, time_period):
    list_of_dframes_pol = []
    for pollution in pollution_types:
        time_period_pol = time_period.loc[time_period["City"] == city].reset_index(drop=True)
        time_period_pol = time_period_pol.loc[time_period_pol["Specie"] == f"{pollution}"].sort_values("Date").reset_index(drop=True).rename(columns={"median": f"{pollution}"})
        list_of_dframes_pol.append(time_period_pol)
    return list_of_dframes_pol
    


In [431]:
def pollution_combiner(dataframes, columns):
    pollution_types = ["o3", "pm25", "pm10", "no2"]
    dataframe = {}
    for pollution in range(len(dataframes)):
        if pollution == 0:
            dataframe = dataframes[0].join(dataframes[1][[pollution_types[1]]])
        if pollution > 1:
            dataframe = dataframe.join(dataframes[pollution][[pollution_types[pollution]]])
    dataframe = dataframe[columns]
    return dataframe


In [432]:
columns = ["Date", "o3", "pm25", "pm10", "no2"]

Q12019_frankfurt = get_pollution_data("Wiesbaden", Q12019)
Q22019_frankfurt = get_pollution_data("Wiesbaden", Q22019)
Q32019_frankfurt = get_pollution_data("Wiesbaden", Q32019)
Q42019_frankfurt = get_pollution_data("Wiesbaden", Q42019)
Q2020_frankfurt = get_pollution_data("Wiesbaden", Q2020)

Q12019_frankfurt = pollution_combiner(Q12019_frankfurt, columns)
Q22019_frankfurt = pollution_combiner(Q22019_frankfurt, columns)
Q32019_frankfurt = pollution_combiner(Q32019_frankfurt, columns)
Q42019_frankfurt = pollution_combiner(Q42019_frankfurt, columns)
Q2020_frankfurt = pollution_combiner(Q2020_frankfurt, columns)

In [435]:
dataframe = Q12019_frankfurt.append(Q22019_frankfurt).append(Q32019_frankfurt).append(Q42019_frankfurt).append(Q2020_frankfurt).reset_index(drop=True)

In [447]:
def dataframe_for_timeperiod(dataframe, time_period_min, time_period_max):
    dataframe_time_period = dataframe.loc[dataframe["Date"] >= time_period_min].reset_index(drop=True)
    dataframe_time_period = dataframe_time_period.loc[dataframe_time_period["Date"] <= time_period_max].reset_index(drop=True)
    return dataframe_time_period


dataframe_for_timeperiod = dataframe_for_timeperiod(dataframe, "2019-06-16", "2020-06-16")

In [448]:
dataframe_for_timeperiod.to_json("../data/clean_pollutiondata.json")